### Take Home Assignment # 1 Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#credits">Credits Dataset</a></li>
<li><a href="#movies">Movies Dataset</a></li>
<li><a href="#ratings">Ratings Dataset</a></li>
<li><a href="#sql">SQL for Franchises</a></li>
</ul>

### <a id='intro'></a>Task #1: Franchise Discovery
> 1. Write SQL query against Kaggle “Movies Dataset” use these files only: movies_metadata.csv, credits.csv, ratings.csv to discover all franchise in the dataset.
> 2. Definition of “franchise”: titles that share the same “cast-character” pair(s)

In [28]:
import pyspark
import ast
from pyspark.sql import functions as f
from pyspark.sql import Window
from pyspark.sql.functions import from_json, col, udf, lit, when, row_number, regexp_extract, split, explode
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, \
                                IntegerType, TimestampType, FloatType, DecimalType


spark = pyspark.sql.SparkSession.builder.appName('movies-dataset').getOrCreate()

In [3]:
import numpy as np 
import pandas as pd 
from pandas.io.json import json_normalize 

In [4]:
spark

### <a id='credits'></a> Credits Dataset

In [5]:
credits_pd = pd.read_csv('/Users/mariamjoan/Notebooks/Movies-Dataset/credits.csv')
credits_pd.head(1)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862


In [6]:
credits = spark.read.option("header", True)\
            .option("ignoreLeadingWhiteSpace", True)\
            .option("quote", '"')\
            .option("escape", '"')\
            .csv("/Users/mariamjoan/Notebooks/Movies-Dataset/credits.csv", inferSchema = False)

In [7]:
credits.printSchema()

root
 |-- cast: string (nullable = true)
 |-- crew: string (nullable = true)
 |-- id: string (nullable = true)



In [8]:
credits.select('cast').show(2, truncate=True)

+--------------------+
|                cast|
+--------------------+
|[{'cast_id': 14, ...|
|[{'cast_id': 1, '...|
+--------------------+
only showing top 2 rows



In [9]:
credits.dtypes

[('cast', 'string'), ('crew', 'string'), ('id', 'string')]

In [10]:
credits.count()

45476

In [11]:
def parser(col_str):  
    b = ast.literal_eval(col_str)
    tupe = []
    for bb in b:
        tupe.append( (bb['character'], bb['name']) )
    return tupe

In [12]:
parser_udf = udf(lambda z: parser(z), ArrayType(StructType([StructField("character", StringType(), True),\
                                  StructField("name", StringType(), True)])))

In [111]:
characters = credits.withColumn("cast_", explode(parser_udf(col("cast"))))\
                    .withColumn("id2", credits.id.cast(IntegerType()))\
                    .filter("cast_.character is not null and cast_.character <> ''")

In [105]:
characters.printSchema()

root
 |-- cast: string (nullable = true)
 |-- crew: string (nullable = true)
 |-- id: string (nullable = true)
 |-- cast_: struct (nullable = true)
 |    |-- character: string (nullable = true)
 |    |-- name: string (nullable = true)
 |-- id2: integer (nullable = true)



In [106]:
characters.select("cast_.character", "cast_.name", "id2").show(5, truncate=False)

+-----------------------+-------------+---+
|character              |name         |id2|
+-----------------------+-------------+---+
|Woody (voice)          |Tom Hanks    |862|
|Buzz Lightyear (voice) |Tim Allen    |862|
|Mr. Potato Head (voice)|Don Rickles  |862|
|Slinky Dog (voice)     |Jim Varney   |862|
|Rex (voice)            |Wallace Shawn|862|
+-----------------------+-------------+---+
only showing top 5 rows



### <a id='ratings'></a> Ratings Dataset

In [16]:
ratings_pd = pd.read_csv('/Users/mariamjoan/Notebooks/Movies-Dataset/ratings.csv')
ratings_pd.head(1)

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529


In [17]:
ratings = spark.read.option("header", True).csv("/Users/mariamjoan/Notebooks/Movies-Dataset/ratings.csv") 

In [18]:
ratings.dtypes

[('userId', 'string'),
 ('movieId', 'string'),
 ('rating', 'string'),
 ('timestamp', 'string')]

In [19]:
ratings_casted = ratings.withColumn("rating2", ratings.rating.cast(FloatType())) \
                        .withColumn("id2", ratings.movieId.cast(IntegerType()))

In [20]:
ratings_casted.select("rating2", "Id2").show(5, truncate=False)

+-------+----+
|rating2|Id2 |
+-------+----+
|1.0    |110 |
|4.5    |147 |
|5.0    |858 |
|5.0    |1221|
|5.0    |1246|
+-------+----+
only showing top 5 rows



In [21]:
ratings_casted.printSchema()

root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- rating2: float (nullable = true)
 |-- id2: integer (nullable = true)



In [22]:
ratings_casted.dtypes

[('userId', 'string'),
 ('movieId', 'string'),
 ('rating', 'string'),
 ('timestamp', 'string'),
 ('rating2', 'float'),
 ('id2', 'int')]

### <a id='movies'></a> Movies Dataset

In [23]:
movies_pd = pd.read_csv('/Users/mariamjoan/Notebooks/Movies-Dataset/movies_metadata.csv', low_memory=False)
movies_pd.popularity.head(1)

0    21.946943
Name: popularity, dtype: object

In [24]:
movies = spark.read.option("header", True)\
            .csv("/Users/mariamjoan/Notebooks/Movies-Dataset/movies_metadata.csv")

In [25]:
movies.printSchema()

root
 |-- adult: string (nullable = true)
 |-- belongs_to_collection: string (nullable = true)
 |-- budget: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- homepage: string (nullable = true)
 |-- id: string (nullable = true)
 |-- imdb_id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- poster_path: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- revenue: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- spoken_languages: string (nullable = true)
 |-- status: string (nullable = true)
 |-- tagline: string (nullable = true)
 |-- title: string (nullable = true)
 |-- video: string (nullable = true)
 |-- vote_average: string (nullable = true)
 |-- vote_count: string (nu

In [26]:
movies.show(1, truncate=True)

+-----+---------------------+--------+--------------------+--------------------+---+---------+-----------------+--------------+--------------------+----------+--------------------+--------------------+--------------------+------------+---------+-------+--------------------+--------+-------+---------+-----+------------+----------+
|adult|belongs_to_collection|  budget|              genres|            homepage| id|  imdb_id|original_language|original_title|            overview|popularity|         poster_path|production_companies|production_countries|release_date|  revenue|runtime|    spoken_languages|  status|tagline|    title|video|vote_average|vote_count|
+-----+---------------------+--------+--------------------+--------------------+---+---------+-----------------+--------------+--------------------+----------+--------------------+--------------------+--------------------+------------+---------+-------+--------------------+--------+-------+---------+-----+------------+----------+
|Fal

In [96]:
movies_casted = movies.withColumn("id2", movies.id.cast(IntegerType())) \
                      .withColumn("revenue2", movies.revenue.cast(DecimalType()))\
                      .withColumn("vote_average2", movies.vote_average.cast(FloatType())) \
                      .filter("id2 is not null and id2 <> 0")

In [90]:
movies_casted.select("id2", "original_title", "revenue2", "vote_average2").show(5, truncate=True)

+-----+--------------------+---------+-------------+
|  id2|      original_title| revenue2|vote_average2|
+-----+--------------------+---------+-------------+
|  862|           Toy Story|373554033|          7.7|
| 8844|             Jumanji|262797249|          6.9|
|15602|    Grumpier Old Men|        0|          6.5|
|31357|   Waiting to Exhale|     null|         null|
|11862|Father of the Bri...| 76578911|          5.7|
+-----+--------------------+---------+-------------+
only showing top 5 rows



In [91]:
movies_casted.dtypes

[('adult', 'string'),
 ('belongs_to_collection', 'string'),
 ('budget', 'string'),
 ('genres', 'string'),
 ('homepage', 'string'),
 ('id', 'string'),
 ('imdb_id', 'string'),
 ('original_language', 'string'),
 ('original_title', 'string'),
 ('overview', 'string'),
 ('popularity', 'string'),
 ('poster_path', 'string'),
 ('production_companies', 'string'),
 ('production_countries', 'string'),
 ('release_date', 'string'),
 ('revenue', 'string'),
 ('runtime', 'string'),
 ('spoken_languages', 'string'),
 ('status', 'string'),
 ('tagline', 'string'),
 ('title', 'string'),
 ('video', 'string'),
 ('vote_average', 'string'),
 ('vote_count', 'string'),
 ('id2', 'int'),
 ('revenue2', 'decimal(10,0)'),
 ('vote_average2', 'float')]

### Drop Columns & Duplicates

In [32]:
characters = characters.drop("cast", "crew", "id")

In [33]:
ratings_casted = ratings_casted.drop("userId", "movieId", "rating", "timestamp")

In [97]:
movies_casted = movies_casted.drop("popularity", "popularity2", "id").drop_duplicates()

### Set DataFrames to Temp Tables to Query

In [112]:
movies_casted.registerTempTable("movies")
ratings_casted.registerTempTable("ratings")
characters.registerTempTable("cast")

### <a id='sql'></a> SQL for Franchises

In [125]:
spark.sql("""   
select * from movies where id2 = 15028
""").toPandas()

,adult,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,poster_path,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,id2,revenue2,vote_average2
0,False,None,26000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 10751...",None,tt0157472,en,Clockstoppers,"Until now, Zak Gibbs' greatest challenge has b...",/xGhDPrBz9mJN8CsIjA23jQSd3sc.jpg,...,"[{'iso_639_1': 'cs', 'name': 'Český'}, {'iso_6...",Released,"The adventure of a lifetime, in a few mere sec...",Clockstoppers,False,4.9,90,15028,38793283,4.9
1,False,None,26000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 10751...",None,tt0157472,en,Clockstoppers,"Until now, Zak Gibbs' greatest challenge has b...",/xGhDPrBz9mJN8CsIjA23jQSd3sc.jpg,...,"[{'iso_639_1': 'cs', 'name': 'Český'}, {'iso_6...",Released,"The adventure of a lifetime, in a few mere sec...",Clockstoppers,False,4.9,89,15028,38793283,4.9


In [99]:
spark.sql("""   
select id2, count(*) from movies group by 1 order by 2 desc
""").show()

+------+--------+
|   id2|count(1)|
+------+--------+
| 15028|       2|
| 85910|       1|
| 46909|       1|
| 30972|       1|
|161816|       1|
| 23619|       1|
|  7131|       1|
|  9488|       1|
|293126|       1|
|223566|       1|
|101887|       1|
| 43821|       1|
| 63066|       1|
|292214|       1|
| 10022|       1|
| 43386|       1|
| 37845|       1|
| 63579|       1|
|246011|       1|
|409082|       1|
+------+--------+
only showing top 20 rows



In [56]:
spark.sql("""   
select id2, count(*) from ratings group by 1 order by 2 desc
""").show()

+----+--------+
| id2|count(1)|
+----+--------+
| 356|   91921|
| 318|   91082|
| 296|   87901|
| 593|   84078|
|2571|   77960|
| 260|   77045|
| 480|   74355|
| 527|   67662|
| 110|   66512|
|   1|   66008|
|1210|   62714|
| 589|   61836|
|1196|   61672|
|2959|   60024|
|1198|   59693|
|  50|   59271|
|2858|   57879|
| 150|   57416|
| 780|   57232|
| 858|   57070|
+----+--------+
only showing top 20 rows



In [113]:
spark.sql("""   
select cast_.character, count(*) from cast group by 1 order by 2 desc
""").show()

+--------------------+--------+
|           character|count(1)|
+--------------------+--------+
|             Himself|   11437|
|             Herself|    3073|
|              Doctor|     663|
|              Dancer|     583|
|            Narrator|     562|
|            Reporter|     545|
|             himself|     542|
|               Nurse|     499|
|        (uncredited)|     472|
|                Anna|     464|
|Additional Voices...|     449|
|                Paul|     446|
|                 Sam|     443|
|           Policeman|     419|
|               Maria|     415|
|               Frank|     414|
|               Sarah|     407|
|           Bartender|     398|
|Himself (archive ...|     391|
|               David|     390|
+--------------------+--------+
only showing top 20 rows



In [124]:
spark.sql("""   
with characters as (
    select
         c.id2
         , c.cast_['character'] as character
         , c.cast_['name'] as talent
        ,count(c.id2) over(partition by c.cast_['character'], c.cast_['name']) franchise_pair 
    from cast c   
    where c.cast_['character'] not in ('Himself', 'Herself', 'himself', 'herself')
    
), 
ratings_agg (
    select 
         r.id2
        ,round(avg(r.rating2), 2) avg_rating_per_title
    from ratings r
    group by 1
)
select
     m.id2
    ,c.character
    ,c.talent
    ,m.original_title
    ,m.imdb_id 
    ,r.avg_rating_per_title
    ,c.franchise_pair
from characters c
    join movies m on c.id2 = m.id2
    join ratings_agg r on m.id2 = r.id2
where c.franchise_pair >= 2
order by 6 desc
""").show(50)

+------+--------------------+--------------------+--------------------+---------+--------------------+--------------+
|   id2|           character|              talent|      original_title|  imdb_id|avg_rating_per_title|franchise_pair|
+------+--------------------+--------------------+--------------------+---------+--------------------+--------------+
|119593|               Extra|      Leoda Richards|A Fever In The Blood|tt0054873|                 4.5|             2|
| 96702|               Nurse|         Mary Gordon|The Power and the...|tt0024465|                 4.5|             2|
|129360|          Usain Bolt|          Usain Bolt|Usain Bolt, La Lé...|tt2583020|                 4.5|             2|
|106113|              Warden|       George Irving| Don't Tell the Wife|tt0028799|                 4.5|             2|
|   318|                 Joe|       Richard Edson|The Million Dolla...|tt0120753|                4.43|             2|
| 91007|               Shing|     Corey Yuen Kwai|Dou Si